In [2]:
import json
import psycopg2
import psycopg2.extras as extras
import paramiko
from xml.etree.ElementTree import ElementTree, tostring
import re
from urllib.parse import unquote


In [7]:
host = '82.198.160.11'
user = 'lnw1nekit'
secret = 'JSajkMGOGfOW'
port = 2224

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
stdin, stdout, stderr = client.exec_command('cat /var/log/raw_input.log')
data = stdout.read() + stderr.read()
# lines = stdout.readlines()
# print(lines)
#cat /var/log/raw_input.log
print(data)
client.close()


b'POST / HTTP/1.1\r\nAccept: application/json, text/javascript, */*; q=0.01\r\nContent-Type: application/x-www-form-urlencoded; charset=UTF-8\r\nHost: 82.198.160.11:8080\r\nContent-Length: 572\r\nExpect: 100-continue\r\nConnection: Close\r\n\r\njsondata%5Buuid%5D=87a82002-5b84-4da2-8fd2-12270bcaf0a1&jsondata%5Btype%5D=7&jsondata%5Btime%5D=2024-07-10 13:54:05&jsondata%5Bsourceid%5D=&jsondata%5Btargetid%5D=&jsondata%5Bstate%5D=2&jsondata%5Bfile%5D=&jsondata%5Bsourcename%5D=&jsondata%5Btargetname%5D=&jsondata%5Bsourceext%5D=&jsondata%5Btargetext%5D=&jsondata%5Bextdata%5D=&jsondata%5Btaskid%5D=b6deb980-ccfe-46e7-b672-13e395551b7f&jsondata%5Bplaylistid%5D=&jsondata%5Bishost%5D=0&jsondata%5Bsourcepanel%5D=0&jsondata%5Btargetpanel%5D=0&jsondata%5Btasktype%5D=0&jsondata%5Bdeviceinfo%5D=&jsondata%5Busertaskinfo%5D=POST / HTTP/1.1\r\nAccept: application/json, text/javascript, */*; q=0.01\r\nContent-Type: application/x-www-form-urlencoded; charset=UTF-8\r\nHost: 82.198.160.11:8080\r\nContent-Leng

In [ ]:
import psycopg2.extras

In [ ]:
from flask import Flask, request, jsonify
import urllib.parse
import re
import json
import psycopg2
from psycopg2 import sql
import uuid
from datetime import datetime
 
app = Flask(__name__)


def decode_and_clean(data):
    print(f"Исходные данные: {data}")
    # Декодирование URL-кодированной строки
    decoded_str = urllib.parse.unquote(data)
    print(f"Декодированные данные: {decoded_str}")

    # Регулярное выражение для извлечения сегментов с uuid
    segments = re.split(r'(?=jsondata\[uuid\]=)', decoded_str)


    result_list = []

    for segment in segments:
        if 'jsondata[uuid]=' not in segment:
            continue

        segment = re.sub(r'jsondata\[([^\]]+)\]', r'\1', segment)

        # Разделение сегмента на пары ключ=значение
        pairs = segment.split('&')
        result = {
            "uuid": "",
            "type": "",
            "time": "",
            "sourceid": "",
            "targetid": "",
            "state": "",
            "file": "",
            "sourcename": "",
            "targetname": "",
            "sourceext": "",
            "targetext": "",
            "extdata": "",
            "taskid": "",
            "playlistid": "",
            "ishost": "",
            "sourcepanel": "",
            "targetpanel": "",
            "tasktype": "",
            "deviceinfo": "",
            "usertaskinfo": ""
        }

        should_skip = False

        for pair in pairs:
            if '=' in pair:
                key, value = pair.split('=', 1)
                value = value.replace('+', ' ')
                if key == 'usertaskinfo' and 'GET' in value:
                    # Если GET запрос, пропускаем сегмент
                    should_skip = True
                    break
                if key in result:
                    if key == 'usertaskinfo':
                        # Извлечение данных до первой новой строки
                        value = value.split('\r\n')[0]
                    result[key] = value

        if not should_skip:
            result_list.append(result)

    return result_list


def save_to_db(data):
    # Подключение к базе данных
    conn = psycopg2.connect(
        dbname="skit_json",
        user="lnw1nekit",
        password="...",  # Замените на ваш реальный пароль
        # Замените на ваш реальный хост (например, "localhost")
        host="...",
        port="..."       # Замените на ваш реальный порт (например, "5432")
    )
    cursor = conn.cursor()

    # SQL-запрос для вставки данных
    insert_query = sql.SQL("""
        INSERT INTO data_json (
            uuid, type, time, sourceid, targetid, state, file, sourcename, targetname,
            sourceext, targetext, extdata, taskid, playlistid, ishost, sourcepanel, 
            targetpanel, tasktype, deviceinfo, usertaskinfo
        ) VALUES (
            %(uuid)s, %(type)s, %(time)s, %(sourceid)s, %(targetid)s, %(state)s, %(file)s,
            %(sourcename)s, %(targetname)s, %(sourceext)s, %(targetext)s, %(extdata)s, %(taskid)s,
            %(playlistid)s, %(ishost)s, %(sourcepanel)s, %(targetpanel)s, %(tasktype)s, %(deviceinfo)s, %(usertaskinfo)s
        )
    """)

    for item in data:
        # Преобразование данных
        try:
            item['uuid'] = uuid.UUID(item['uuid'])
            item['type'] = int(item['type'])
            item['time'] = datetime.strptime(item['time'], '%Y-%m-%d %H:%M:%S')
            item['state'] = int(item['state'])
            item['taskid'] = uuid.UUID(item['taskid'])
            item['ishost'] = int(item['ishost'])
            item['sourcepanel'] = int(item['sourcepanel'])
            item['targetpanel'] = int(item['targetpanel'])

            # Преобразование targetext: либо пустая строка, либо массив [{}]
            if item['targetext'] == '':
                item['targetext'] = ''
            elif isinstance(item['targetext'], str):
                targetext_value = json.loads(item['targetext'])
                if targetext_value == [{}]:
                    item['targetext'] = json.dumps([{}])
                else:
                    raise ValueError("Invalid value for targetext")

        except (ValueError, TypeError, json.JSONDecodeError) as e:
            print(f"Error converting data: {e}")
            continue

        # Вставка данных в таблицу
        cursor.execute(insert_query, item)

    print("Данные в бд")

    # Фиксация изменений и закрытие подключения
    conn.commit()
    cursor.close()
    conn.close()


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        print(request.data)
        try:
            if request.is_json:
                data = request.get_json()
                print(f"Received JSON data: {data}")
            else:
                data = request.data.decode('utf-8')
                print(f"Received string data: {data}")
                data = decode_and_clean(data)
            print(f"Processed data: {data}")
            save_to_db(data)
            return jsonify(data), 200
        except Exception as e:
            print(f"Error processing data: {e}")
            return "Error processing data", 500
    return "Hello, World", 200


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


In [5]:
data1 = "jsondata[uuid]=123&jsondata[type]=example"

In [6]:
import urllib.parse
import re
import json


def decode_and_clean(data):
    # Декодирование URL-кодированной строки
    decoded_str = urllib.parse.unquote(data)

    # Очистка строки от невалидных символов
    decoded_str = re.sub(r'[^\x20-\x7E]', '', decoded_str)

    # Регулярное выражение для извлечения сегментов с uuid
    segments = re.split(r'(?=jsondata\[uuid\]=)', decoded_str)

    result_list = []

    for segment in segments:
        if 'jsondata[uuid]=' not in segment:
            continue

        segment = re.sub(r'jsondata\[([^\]]+)\]', r'\1', segment)

        # Разделение сегмента на пары ключ=значение
        pairs = segment.split('&')

        result = {
            "uuid": "",
            "type": "",
            "time": "",
            "sourceid": "",
            "targetid": "",
            "state": "",
            "file": "",
            "sourcename": "",
            "targetname": "",
            "sourceext": "",
            "targetext": [{}],
            "extdata": "",
            "taskid": "",
            "playlistid": "",
            "ishost": "",
            "sourcepanel": "",
            "targetpanel": "",
            "tasktype": "",
            "deviceinfo": "",
            "usertaskinfo": ""
        }

        should_skip = False

        for pair in pairs:
            if '=' in pair:
                key, value = pair.split('=', 1)
                value = value.replace('+', ' ')

                if key == 'usertaskinfo':
                    # Извлечение только части строки "POST/HTTP/любое число"
                    match = re.search(r'POST\s+.*?HTTP/\d+(\.\d+)?', value)
                    if match:
                        result[key] = match.group(0)
                    else:
                        result[key] = ""

                elif 'GET' in value:
                    # Если GET запрос, пропускаем сегмент
                    should_skip = True
                    break

                elif key in result:
                    if key == 'targetext':
                        if value:
                            try:
                                value = json.loads(value)
                            except json.JSONDecodeError:
                                value = [{}]
                        else:
                            value = [{}]
                    result[key] = value

        # Если сегмент не нужно пропускать и содержит данные, добавляем в результат
        if not should_skip and result['uuid']:
            result_list.append(result)

    return result_list


# # Пример данных для тестирования
# data = 'jsondata%5Bsourceid%5D=9990010&jsondata%5Btargetid%5D=1&jsondata%5Bstate%5D=2&jsondata%5Bfile%5D=&jsondata%5Bsourcename%5D=Visotski1&jsondata%5Btargetname%5D=Terminal1(1)&jsondata%5Bsourceext%5D=&jsondata%5Btargetext%5D=[{}]&jsondata%5Bextdata%5D=&jsondata%5Btaskid%5D=b6deb980-ccfe-46e7-b672-13e395551b7f&jsondata%5Bplaylistid%5D=1_visotski_visota.mp3&jsondata%5Bishost%5D=0&jsondata%5Bsourcepanel%5D=0&jsondata%5Btargetpanel%5D=0&jsondata%5Btasktype%5D=&jsondata%5Bdeviceinfo%5D=&jsondata%5Busertaskinfo%5D=POST / HTTP/1.1&jsondata%5Buuid%5D=ce4b088a-d74c-40c0-bfb4-2a463b553525&jsondata%5Btype%5D=4&jsondata%5Btime%5D=2024-07-10 13:54:05'

# Декодирование и очистка данных
cleaned_data = decode_and_clean(data1)

# Преобразование в JSON строку с форматированием
formatted_json = json.dumps(cleaned_data, indent=4, ensure_ascii=False)

# Вывод на экран
print(formatted_json)

# Если нужно сохранить результат в файл
output_file = 'formatted_output.json'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(formatted_json)

print(f'Данные записаны в файл: {output_file}')


[
    {
        "uuid": "123",
        "type": "example",
        "time": "",
        "sourceid": "",
        "targetid": "",
        "state": "",
        "file": "",
        "sourcename": "",
        "targetname": "",
        "sourceext": "",
        "targetext": [
            {}
        ],
        "extdata": "",
        "taskid": "",
        "playlistid": "",
        "ishost": "",
        "sourcepanel": "",
        "targetpanel": "",
        "tasktype": "",
        "deviceinfo": "",
        "usertaskinfo": ""
    }
]
Данные записаны в файл: formatted_output.json


In [ ]:
# Удаление невалидных символов (все, что не соответствует ASCII)
cleaned_str = re.sub(r'[^\x20-\x7E]', '', decoded_str)

# Разделение строки на словари по основным ключевым словам, которые можно использовать как разделители
segments = re.split(
    r'(?<=jsondata\[time\]=\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})&jsondata\[', cleaned_str)
